<a href="https://colab.research.google.com/github/chacha86/pythonai2/blob/main/%EC%9D%B4%EC%A0%84_%EB%9E%AD%ED%82%B9_%EB%89%B4%EC%8A%A4_%EC%8A%A4%ED%81%AC%EB%9E%A9_%EB%AA%A8%EB%93%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 언론사 이름과 링크 가져오기
## 파이썬 웹 요청 라이브러리
import requests


## 파이썬이 브라우저인 척 하기 위함
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

## 네이버 랭킹 뉴스 웹페이지 요청. 성공하면 응답이옴(r)
r = requests.get('https://news.naver.com/main/ranking/popularDay.naver', headers=headers)


## 응답 정보에는 많은 게 있지만 우린 text가져가면 된다. text는 단순 문자열이다.
ranking_news_html = r.text

## 뷰티풀숩으로 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(ranking_news_html, 'html.parser')

# 하나만 선택 -> select_one
# 여러개 선택 -> select

box_list = soup.select("._officeCard .rankingnews_box")


journal_list = []

for box in box_list :
  atag = box.select_one('a')

  # 태그를 얻었을 때
  # 1. 텍스트
  # 2. 속성
  journal_link = atag['href']
  journal_name = atag.select_one('strong').text

  journal_dic = {
      'name' : journal_name,
      'link' : journal_link
  }

  journal_list.append(journal_dic)


def get_journal_by_name(name) :

  for journal in journal_list:
    if journal['name'] == name :
      return journal
  return None


def get_page_by_url(url) :
  from bs4 import BeautifulSoup
  import requests

  headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
  }
  r = requests.get(url, headers=headers)
  soup = BeautifulSoup(r.text, 'html.parser')

  return soup


def get_rankingnews_by_page(page) :
  thumb_list = page.select('.as_thumb')

  news_list = []

  for thumb in thumb_list :
    link = thumb.select_one('a')['href'] # 본문링크
    rank = thumb.select_one('em').text # 랭킹
    title = thumb.select_one(".list_content strong").text  # 제목
    tmp = thumb.select_one(".list_content .list_view").text #  조회수,
    view = tmp[4:].strip()

    news_dic = {
        'link' : link,
        'rank' : rank,
        'title' : title,
        'view' : view
    }

    news_list.append(news_dic)

  return news_list


import json
# 4. 파이썬 오브젝트를 json으로 저장해주는 함수
def save_to_json(filepath, obj) :
  with open(filepath, 'w') as f :
    json_str = json.dumps(obj, ensure_ascii=False)
    f.write(json_str)

# 5. json 파일을 파이썬 오브젝트로 불러오는 함수
def load_json(filepath) :
  with open(filepath, 'r') as f:
    json_str = f.read()
    return json.loads(json_str)
